## CAPE FORUM Surrogate models

This short example is intended to create to create and assess a set of surrogate models

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pytictoc import TicToc
from sklearn.metrics import mean_squared_error, r2_score, max_error, mean_absolute_error
from smt.sampling_methods import LHS
from smt.surrogate_models import LS, KPLS, KRG, KPLSK, GEKPLS, IDW, RBF

In [2]:
# In my model I have 6 inputs (ndim) and I'm starting with 12 samples (ndoe) 

In [3]:
ndim = 6
ndoe = 12
T_s,T_a,D_a,D_s,Tr_a,Tr_s = [40,60],[90,105],[15,20],[5,12.5],[0.6,1.2],[0.6,1.8] # the range of the input parameters
limits = [T_s,T_a,D_a,D_s,Tr_a,Tr_s]

In [4]:
# I load my 'Aspen' file 

In [5]:
filename = 'Aspen/my_aspen.pkl'

In [6]:
with open(filename, "rb") as f:
   org_model = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator GaussianMixture from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# The list of possible surrogate models I'm going to use and compare
surrogate_models = ['LS', 'KPLS', 'KRG', 'KPLSK', 'GEKPLS', 'IDW', 'RBF']

In [8]:
# These are the non-explored space points, in which I want to build a surrogate
ntest = 1000
sampling = LHS(xlimits=np.array(limits), criterion='ese', random_state=1992)
xtest = sampling(ntest)

In [9]:
def initial_DOE():
    # Construction of the initial sampling
    np.random.seed(1992)
    sampling = LHS(xlimits=np.array(limits), criterion='ese', random_state=1992)
    xt = sampling(ndoe)
    # Compute the outputs
    yt = org_model.predict_values(xt)

    return xt,yt

In [10]:
def build_surrogate(xt,yt,xtest,model_type):
    if model_type == 'KPLS':
        t = KPLS(n_comp=6, theta0=[1e-2] * ndim, print_prediction=False, corr='abs_exp')
    if model_type == 'KPLSK':
        t = KPLS(n_comp=6, theta0=[1e-2] * ndim, print_prediction=False, corr='abs_exp')
    if model_type == 'GEKPLS':
        t = KPLS(n_comp=6, theta0=[1e-2] * ndim, print_prediction=False, corr='abs_exp')
    elif model_type == 'KRG':
        t = KRG(theta0=[1e-2] * ndim, print_prediction=False)
    elif model_type == 'LS':
        t = LS(print_prediction=False)
    elif model_type == 'RBF':
        t = RBF(print_prediction=False,poly_degree = 0)
    elif model_type == 'IDW':
        t = IDW(print_prediction=False)

    t.set_training_values(xt, yt[:, 0])
    t.train()
    y = t.predict_values(xtest)

    return y,t

In [11]:
def load_surrogate(model, xtest):
    # For loading the model
    y = model.predict_values(xtest)
    return y

In [12]:
def PRF_learning_function(y,xt,yt):

    # Define a distance-based function

    x_test_hr = []
    for t in xtest: # 200 potential points
        aux = []
        for xi in xt: # for all the DOE
            aux.append(np.linalg.norm(t-xi))
        dist_min = min(aux)
        x_test_hr.append(dist_min)

    aux = []
    for xi in xt:
        for xj in xt:
            aux.append(np.linalg.norm(xtest[1] - xi))
    dist_max = max(aux)

    x_test_hr = x_test_hr/dist_max
    index_max = np.argmax(x_test_hr)
    xtest[index_max] # the new one to insert in the sampling

    # Define a Potential risk-function
    tau = 0.5
    x_test_ht = []
    for _,t in enumerate(xtest):
        if y[_] >= 0:
            r = y[_]
        else:
            r = - y[_]
        x_test_ht.append(np.exp(-tau*r/((max(yt)-0))).tolist()[0])

    x_test_PRF = x_test_hr*x_test_ht
    index_next = np.argmax(x_test_PRF)
    xtest[index_next] # the new one to insert in the sampling

    return xtest[index_next]

In [13]:
def dist_PRF(y,xt,yt):

    # Define a distance-based function

    x_test_hr = []
    for t in xtest: # 200 potential points
        aux = []
        for xi in xt: # for all the DOE
            aux.append(np.linalg.norm(t-xi))
        dist_min = min(aux)
        x_test_hr.append(dist_min)

    aux = []
    for xi in xt:
        for xj in xt:
            aux.append(np.linalg.norm(xtest[1] - xi))
    dist_max = max(aux)

    x_test_hr = x_test_hr/dist_max
    index_max = np.argmax(x_test_hr)
    xtest[index_max] # the new one to insert in DOE

    # Define a Potential risk-function
    tau = 0.5
    x_test_ht = []
    for _,t in enumerate(xtest): # 200 potential points
        if y[_] >= 0:
            r = y[_]
        else:
            r = - y[_]
        x_test_ht.append(np.exp(-tau*r/((max(yt)-0))).tolist()[0])

    x_test_PRF = x_test_hr*x_test_ht
    index_next = np.argmax(x_test_PRF)
    xtest[index_next] # the new one to insert in DOE

    return x_test_PRF

In [14]:
def stop_criteria(xt,yt,model):

    Smc = 1000
    sampling = LHS(xlimits=np.array(limits), criterion='ese', random_state=1992)
    x_smc = sampling(Smc)
    epsilon = []
    y_smc = load_surrogate(model, x_smc)
    s2_smc = dist_PRF(y_smc,xt,yt)*dist_PRF(y_smc,xt,yt)#It is a distance for each point based on the PRF (not really variance)
    for _, t in enumerate(x_smc):
        ep = np.sqrt(s2_smc[_]) / y_smc[_]
        epsilon.append(ep)

    return max(epsilon)

In [15]:
def stop_criteria_beta_bound(model):

    Smc = 1000
    sampling = LHS(xlimits=np.array(limits), criterion='ese', random_state=1992)
    x_smc = sampling(Smc)

    y_smc = load_surrogate(model, x_smc)
    s2_smc = dist_PRF(y_smc,xt,yt)*dist_PRF(y_smc,xt,yt)#It is a distance for each point based on the PRF (not really variance)
    beta = []
    beta_plus, beta_minus, miu_GMM_plus, miu_GMM_minus = [], [], [], []
    for _, t in enumerate(x_smc):
        vi = s2_smc[_] + (y_smc[_]) ** 2
        vi_plus = s2_smc[_] + (y_smc[_] - 2 * np.sqrt(s2_smc[_])) ** 2
        miu_GMM_plus.append(y_smc[_] - 2 * np.sqrt(s2_smc[_]))
        miu_GMM_minus.append(y_smc[_] + 2 * np.sqrt(s2_smc[_]))
        vi_minus = s2_smc[_] + (y_smc[_] + 2 * np.sqrt(s2_smc[_])) ** 2
        beta.append(vi)
        beta_plus.append(vi_plus)
        beta_minus.append(vi_minus)

    v_GMM = (np.sum(beta) / len(y_smc)) - (np.average(y_smc)) ** 2
    v_GMM_plus = (np.sum(beta_plus) / len(y_smc)) - (np.average(miu_GMM_plus)) ** 2
    v_GMM_minus = (np.sum(beta_minus) / len(y_smc)) - (np.average(miu_GMM_minus)) ** 2
    beta = np.average(y_smc) / np.sqrt(v_GMM)
    beta_plus = np.average(miu_GMM_plus) / np.sqrt(v_GMM_plus)
    beta_minus = np.average(miu_GMM_minus) / np.sqrt(v_GMM_minus)

    beta_bound = np.abs(beta_plus - beta_minus) / beta

    return beta,beta_bound

In [16]:
def stop_criteria_beta_stability(beta_list):
    if len(beta_list) == 1:
        beta_stability = 1
    else:
        beta_stability = np.abs(beta_list[len(beta_list)-1]-beta_list[len(beta_list)-2])/beta_list[len(beta_list)-1]

    return beta_stability

In [ ]:
# Defining the threshold values:
e_th_value = 0.5
bb_th_value = 0.10
bs_th_value = 0.05

In [ ]:
if __name__ == '__main__':
    for model_type in surrogate_models:
        time = TicToc()  # create instance of class
        time.tic()
        xt, yt = initial_DOE()
        e_th, beta_bound, beta_sta = 10,10,10
        store_pf,store_ep,metrics = [],[],[]
        beta_list, beta_bound_list, beta_sta_list = [],[],[]
        
        while e_th > e_th_value or beta_bound > bb_th_value or beta_sta > bs_th_value:
            y, model = build_surrogate(xt, yt, xtest, model_type)
            
            # I want to decide the next point to add -> I define a learning function.
            next_point = PRF_learning_function(y, xt, yt)

            # stopping criteria -> when do I stop adding new points?
            e_th = stop_criteria(xt, yt, model)
            
            # Beta stopping criteria
            beta, beta_bound = stop_criteria_beta_bound(model)  # beta bound value
            beta_list.append(beta)
            beta_sta = stop_criteria_beta_stability(beta_list)  # beta stability value
            beta_sta_list.append(beta_sta)
            beta_bound_list.append(beta_bound)
            
            # epsilon stopping criteria
            if e_th > e_th_value or beta_bound > bb_th_value or beta_sta > bs_th_value:
                xt = np.append(xt, np.reshape(next_point, [1, ndim]), axis=0)
                yt = np.append(yt, org_model.predict_values(np.reshape(next_point, [1, ndim])),axis=0)
                xtest = sampling(ntest)
            print(e_th, beta_bound, beta_sta)
            store_ep.append(e_th)

            y_hat = load_surrogate(model, xt)
            maxerror = max_error(y_hat, yt)
            meanabs_error = mean_absolute_error(y_hat, yt)
            mse = mean_squared_error(y_hat, yt)  # mean square error
            RE = mse ** 0.5 / np.sum(yt ** 2) ** 0.5 * 100  # relative error
            metrics.append([maxerror, meanabs_error, mse, RE])
        
        print(e_th)
        toc = time.tocvalue()
        time = [toc]
        store_pf = [len(xt)]
        
        np.savetxt("Results/epsilon_"+model_type+".csv", store_ep, delimiter=",", fmt="% s")
        np.savetxt("Results/Size_"+model_type+".csv", store_pf, delimiter=",", fmt="% s")
        np.savetxt("Results/time_"+model_type+".csv", time, delimiter=",", fmt="% s")
        np.savetxt("Results/metrics_"+model_type+".csv", metrics, delimiter=",", fmt="% s")
        np.savetxt("Results/beta_bound_"+model_type+".csv", beta_bound_list, delimiter=",", fmt="% s")
        np.savetxt("Results/beta_sta_"+model_type+".csv", beta_sta_list, delimiter=",", fmt="% s")
        np.savetxt("Results/beta_list_"+model_type+".csv", beta_list, delimiter=",", fmt="% s")

___________________________________________________________________________
   
                                    LS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 12
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0014269
[0.06720017] 0.14754182377955055 1
___________________________________________________________________________
   
                                    LS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 13
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0029221
[0.05678126] 0.12020771995917426 0.05074195729107937
___________________________________________________________________________
   
          

[0.03500341] 0.10946596173562738 0.09533531682600546
___________________________________________________________________________
   
                                    LS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 30
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0007138
[0.03732041] 0.0934143679727413 0.02785659823384342
[0.03732041]
___________________________________________________________________________
   
                                   KPLS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 12
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.1971526
[0.06829236] 0.1580793732774734 1
_________________________